In [121]:
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig,RunnableGenerator
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate,MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf
from typing import Iterator,List,AsyncIterator
from langchain_core.runnables import ConfigurableField,ConfigurableFieldSpec
from langchain_community.chat_message_histories import ChatMessageHistory,RedisChatMessageHistory  # new
from langchain_core.chat_history import BaseChatMessageHistory  # new
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory  # new

In [7]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [8]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmGPT=ChatOpenAI(model="gpt-3.5-turbo")

In [76]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        SystemMessagePromptTemplate.from_template(
            template="You're an assistant who's good at {ability}. Respond in 20 words or fewer"
        ),
        MessagesPlaceholder(
            variable_name="history"
        ),
        HumanMessagePromptTemplate.from_template(
            template="{question}"
        )
    ]
)

In [77]:
chain=prompt|llmGemini

<h3>In Memory Chat Message History </h3>

In [78]:
store={}
def getSessionHistory(sessionId:str) -> BaseChatMessageHistory:
    if sessionId not in store:
        store[sessionId]=ChatMessageHistory()
    return store[sessionId]

In [79]:
chainWithMessageHistory=RunnableWithMessageHistory(
                            chain,
                            get_session_history=getSessionHistory,
                            input_messages_key="question",
                            history_messages_key="history"
                        )

In [80]:
sessionID="ritish"

In [81]:
chainWithMessageHistory.invoke(
    input={"ability":"geography",
           "question":"Which is the smallest country in the world?"},
    config={"configurable":{"session_id":sessionID}}
    )

AIMessage(content='Vatican City. \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-f6da379d-5356-4904-8bd9-401a3c2fa535-0', usage_metadata={'input_tokens': 30, 'output_tokens': 4, 'total_tokens': 34})

In [82]:
# Remembers
chainWithMessageHistory.invoke(
    input={"ability":"What?",
           "question":"Where is it located?"},
    config={"configurable":{"session_id":sessionID}}
    )

AIMessage(content='Vatican City is located within Rome, Italy. \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-6068da0f-47ab-4c41-87bf-282a814e4522-0', usage_metadata={'input_tokens': 43, 'output_tokens': 10, 'total_tokens': 53})

In [83]:
store

{'ritish': InMemoryChatMessageHistory(messages=[HumanMessage(content='Which is the smallest country in the world?', additional_kwargs={}, response_metadata={}), AIMessage(content='Vatican City. \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-f6da379d-5356-4904-8bd9-401a3c2fa535-0', usage_metadata={'input_tokens': 30, 'output_tokens': 4, 'total_tokens': 34}), HumanMessage(content='Where is it located?', additional_kwargs={}, response_metadata={}), AIMessage(content='Vatican City is located within Rome, Italy. \

In [89]:
store['ritish'].

InMemoryChatMessageHistory(messages=[HumanMessage(content='Which is the smallest country in the world?', additional_kwargs={}, response_metadata={}), AIMessage(content='Vatican City. \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-f6da379d-5356-4904-8bd9-401a3c2fa535-0', usage_metadata={'input_tokens': 30, 'output_tokens': 4, 'total_tokens': 34}), HumanMessage(content='Where is it located?', additional_kwargs={}, response_metadata={}), AIMessage(content='Vatican City is located within Rome, Italy. \n', additio

In [90]:
# Will not remember due to different sessionID
chainWithMessageHistory.invoke(
    input={"ability":"What?",
           "question":"Where is it located?"},
    config={"configurable":{"session_id":"Tania"}}
    )

AIMessage(content='Please provide me with more context. What are you referring to? \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-8c57faab-4529-499d-90cd-bdcbf84828bb-0', usage_metadata={'input_tokens': 26, 'output_tokens': 13, 'total_tokens': 39})

<h3> Using Configurable Field Spec to include two parameters: userID and conversationID </h3> 

In [92]:
store={}
def getSessionHistory(userID:str, conversationID:str) -> BaseChatMessageHistory:
    if (userID,conversationID) not in store:
        store[(userID,conversationID)]=ChatMessageHistory()
    return store[(userID,conversationID)]

In [107]:
chainWithMessageHistory=RunnableWithMessageHistory(
                            runnable=chain,
                            get_session_history=getSessionHistory,
                            input_messages_key="question",
                            history_messages_key="history",
                            history_factory_config=[
                                ConfigurableFieldSpec(
                                    id="userID",
                                    annotation=str,
                                    name="User ID",
                                    description="Unique Identifier for the User",
                                    default="",
                                    is_shared=True
                                ),
                                ConfigurableFieldSpec(
                                    id="conversationID",
                                    annotation=str,
                                    name="Conversation ID",
                                    description="Unique Identifier for the Conversation",
                                    default="",
                                    is_shared=True
                                )
                            ]
                        )

In [108]:
userID="ritish"
conversationID="1"

In [109]:
chainWithMessageHistory.invoke(
    input={"ability":"geography",
           "question":"Which is the smallest country in the world?"},
    config={"configurable":{"userID":userID,"conversationID":conversationID}}
    )

AIMessage(content='Vatican City is the smallest country in the world. \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'LOW', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-69854e64-1f0f-497b-9382-be3e4eb14345-0', usage_metadata={'input_tokens': 116, 'output_tokens': 11, 'total_tokens': 127})

In [110]:
# Remembers
chainWithMessageHistory.invoke(
    input={"ability":"geography",
           "question":"Where is it located?"},
    config={"configurable":{"userID":userID,"conversationID":conversationID}}
    )

AIMessage(content="It's an enclave within Rome, Italy. \n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-4e3862b4-1070-4703-b8a1-fe7f9ba23275-0', usage_metadata={'input_tokens': 136, 'output_tokens': 10, 'total_tokens': 146})

In [111]:
# Will not remember due to different conversationID
chainWithMessageHistory.invoke(
    input={"ability":"geography",
           "question":"What is it's area?"},
    config={"configurable":{"userID":userID,"conversationID":"2"}}
    )

AIMessage(content="Please provide the location or object you're asking about. \n\n\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-181bde30-7e24-4b0b-b25b-53c8ea0c6c54-0', usage_metadata={'input_tokens': 97, 'output_tokens': 12, 'total_tokens': 109})

<h3>Use with Redis</h3>

In [99]:
REDIS_URL="redis://localhost:6379/0"

In [115]:
def getMessageHistory(sessionID:str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id=sessionID,url=REDIS_URL)

In [116]:
chainWithMessageHistoryRedis=RunnableWithMessageHistory(
    runnable=chain,
    get_session_history=getMessageHistory,
    input_messages_key="question",
    history_messages_key="history"
)

In [117]:
chainWithMessageHistoryRedis.invoke(
    input={"ability":"geography",
           "question":"Which is the smallest country in the world?"},
    config={"configurable":{"session_id":sessionID}}
    )

AIMessage(content='Vatican City is the smallest country in the world. \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'LOW', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-ff9e7df5-bdcc-4a5c-93f3-419b764fc661-0', usage_metadata={'input_tokens': 73, 'output_tokens': 11, 'total_tokens': 84})

In [118]:
# Remembers
chainWithMessageHistoryRedis.invoke(
    input={"ability":"geography",
           "question":"Where is it located?"},
    config={"configurable":{"session_id":sessionID}}
    )

AIMessage(content='Vatican City is located within Rome, Italy. \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-6a1a8a17-6e98-4ba6-8a95-591c0ed42bfc-0', usage_metadata={'input_tokens': 93, 'output_tokens': 10, 'total_tokens': 103})

In [119]:
# Will not remember due to different sessionID
chainWithMessageHistoryRedis.invoke(
    input={"ability":"geography?",
           "question":"Where is it located?"},
    config={"configurable":{"session_id":"Tania"}}
    )

AIMessage(content="Please tell me what you're looking for! 🗺️ \n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-4d10ada1-ade6-4cc7-a932-3738195c7bb8-0', usage_metadata={'input_tokens': 26, 'output_tokens': 13, 'total_tokens': 39})

<h3>Conversational Buffer Memory (Not Working) </h3>

In [128]:
store={}
def getSessionHistory(userID:str, conversationID:str) ->BaseChatMessageHistory:
    if (userID,conversationID) not in store:
        store[(userID,conversationID)]=ChatMessageHistory()
    return store[(userID,conversationID)]

In [129]:
chainWithMessageHistory=RunnableWithMessageHistory(
                            runnable=chain,
                            get_session_history=getSessionHistory,
                            input_messages_key="question",
                            history_messages_key="history",
                            history_factory_config=[
                                ConfigurableFieldSpec(
                                    id="userID",
                                    annotation=str,
                                    name="User ID",
                                    description="Unique Identifier for the User",
                                    default="",
                                    is_shared=True
                                ),
                                ConfigurableFieldSpec(
                                    id="conversationID",
                                    annotation=str,
                                    name="Conversation ID",
                                    description="Unique Identifier for the Conversation",
                                    default="",
                                    is_shared=True
                                )
                            ]
                        )

In [130]:
chainWithMessageHistory.invoke(
    input={"ability":"geography",
           "question":"Which is the smallest country in the world?"},
    config={"configurable":{"userID":userID,"conversationID":conversationID}}
    )

AttributeError: 'ConversationBufferMemory' object has no attribute 'messages'